In [3]:
NAME = 0
OPERATOR = 1
PUSH = 2
POP = 3
ATTR = 4
NUMBER = 5
EOF = 6
EQUAL = 7
NOTEQUAL = 8
LESSTHAN = 9
GREATERTHAN = 10
LESSEQUAL = 11
GREATEREQUAL = 12
DOT = 13
E_SYNTAX = 14
PLUS = 15
MINUS = 16
DIV = 17
MULT = 18
NUMBER = 19
UNARY_PLUS = 20
UNARY_MINUS = 21
POW = 22
FUNC = 23
COMMA = 24
NTYPES = 25


NUM2STR = [
    "NAME",
    "OPERATOR", 
    "PUSH", 
    "POP", 
    "ATTR", 
    "NUMBER", 
    "EOF", 
    "==", 
    "!=", 
    "<", 
    ">", 
    "<=", 
    ">=", 
    ".", 
    "E_SYNTAX", 
    "+", 
    "-", 
    "/", 
    "*",
    "NUMBER",
    "+",
    "-",
    "**",
    "FUNC",
    "COMMA",
]

class Token():
    def __init__(self, typ, val):
        self.type = typ
        self.val = val
    def __repr__(self):
        return "<%s:%s>"%(NUM2STR[self.type], self.val)

class TokenState():
    def __init__(self, s):
        self.i = -1
        self.n = -1
        self.s = s
        self.l = len(s)
        self.tokens = []
        
    def nextc(self):
        self.i += 1
        if self.i >= self.l:
            self.i = self.l
            return '\0'
        return self.s[self.i]
    
    def peekc(self, c):
        return self.peek() == c
            
    def peek(self):
        i = self.i+1
        if i >= self.l:
            return '\0'
        return self.s[i]
    
    def ungetc(self):
        self.i -= 1
        
    def endtok(self):
        return self.s[self.n:self.i+1]
    
    def starttok(self):
        self.n = self.i

def getc(s):
    return s.nextc()

def ungetc(s):
    return s.ungetc()
    
def maybe_name(c):
    o = ord(c)
    if 97 <= o <= 122: return True
    if 65 <= o <= 90 : return True
    if c == "_": return True
    return False

def isnumberstart(c):
    return c in "0123456789-."

def isnumber(c):
    return c in "0123456789."

def parseop(s,c):
    s.starttok()
    c2 = getc(s)
    if c == "=":
        if c2 != "=":
            ungetc(s)
        val = EQUAL
    elif c == "<":
        if c2 == ">":
            val = NOTEQUAL
        elif c2 == "=":
            val = LESSEQUAL
        else:
            ungetc(s)
            val = LESSTHAN
    elif c == ">":
        if c2 == "=":
            val = GREATEREQUAL
        else:
            ungetc(s)
            val = GREATERTHAN
    elif c == ".":
        ungetc(s)
        val = DOT
    elif c == "!":
        if c2 != "=":
            op = E_SYNTAX
            val = E_SYNTAX
    elif c == "+":
        ungetc(s)
        val = PLUS
    elif c == "-":
        ungetc(s)
        val = MINUS
    elif c == "/":
        ungetc(s)
        val = DIV
    elif c == "*":
        if c2 == "*":
            val = POW
        else:
            ungetc(s)
            val = MULT
    return Token(OPERATOR, val)
        
def is_name(c):
    return maybe_name(c) or (48 <= ord(c) <= 57) # include numbers

def tokenize(st):
    s = TokenState(st)
    stack = []
    def token(t,v): stack.append(Token(t,v))
    def lasttype(): return stack[-1].type if stack else NTYPES
    
    level = 0
    while True:
        c = getc(s)
        if c == '\0':
            break  # EOF
        elif c == " ":
            continue
        elif maybe_name(c):               # NAME
            s.starttok()
            while is_name(c):
                c = getc(s)
            ungetc(s)
            name = s.endtok()
            if name in _builtins:
                token(OPERATOR, FUNC)
                token(NUMBER, _builtins[name]._nargs_)
            token(NAME, name)
        elif isnumber(c) or (c == "-" and isnumber(s.peek()) and 
                        (lasttype() == OPERATOR or lasttype() == NTYPES)):  # distinguish operator from unary minus
            s.starttok()
            c = getc(s)         # numbers that start with "-" fail isnumber() without this
            while isnumber(c):
                c = getc(s)
            ungetc(s)
            token(NUMBER, (s.endtok()))
        elif c in "({[":                  # PUSH
            token(PUSH, c)
            level += 1
        elif c in ")}]":                  # POP
            token(POP, c)
            level -= 1
        elif c in "<>=+-/*.":
            tok = parseop(s, c)
            if tok.type == E_SYNTAX:
                raise SyntaxError("Error at position %s"%s.i)
            # handle "5+-4" etc
            if lasttype() == OPERATOR:
                if tok.val == PLUS:
                    tok.val = UNARY_PLUS
                elif tok.val == MINUS:
                    tok.val = UNARY_MINUS
                else:
                    raise SyntaxError("Consecutive token mismatch: %r -> %r"% \
                                    NUM2STR(stack[-1].val), tok.val)
            stack.append(tok)
        elif c == ",":
            token(OPERATOR, COMMA)
        else:
            raise SyntaxError(repr(c))
    if level != 0:
        raise SyntaxError("Mismatched brackets")
    return stack
            
            

In [4]:
import inspect

def _getargs(stack, n):
    if n == -1:
        n = int(stack.pop().val)
    args = [None]*n
    for i in range(n-1, -1, -1):
        args[i] = stack.pop().val
    return args

class Operator():
    def __init__(self, nargs, func, precedence=1, associativity="LEFT"):
        self.n = nargs
        self.f = func
        self.p = precedence
        self.a = associativity
        
    def exec(self, stack):
        args = _getargs(stack, self.n)
        val = self.f(*args)
        stack.append(val)        
    

def _add(a,b):
    return Value(V_NUMBER, a + b)

def _sub(a,b):
    return Value(V_NUMBER, a-b)

def _mult(a,b):
    return Value(V_NUMBER, a * b)

def _div(a,b):
    return Value(V_NUMBER, a/b)

def _valueerror():
    raise UnsupportedOperation()
    
def _getattr(a,b):
    return getattr(a,b)
    
def _pow(a,b):
    return Value(V_NUMBER, a ** b)
    
def _call_function(a, *args):
    f = _builtins[a]
    return f(*args)
    
    
# operators
operators = [Operator(0, _valueerror)] * NTYPES
operators[PLUS]  = Operator(2, _add, 2, "LEFT")
operators[MINUS] = Operator(2, _sub, 2, "LEFT")
operators[MULT]  = Operator(2, _mult, 3, "LEFT")
operators[DIV]   = Operator(2, _div, 3, "LEFT")
operators[DOT]   = Operator(2, _getattr, 1, "LEFT")
operators[POW]   = Operator(2, _pow, 4, "RIGHT")
operators[FUNC]  = Operator(-1, _call_function, 10, "RIGHT")

# builtin functions
# helper decorator
def _builtin(f):
    n = len(inspect.getargs(f.__code__).args)
    f._nargs_ = n
    return f
        
@_builtin
def _ftest(a,b):
    print("ftest(a=%r, b=%r)"%(a,b))
    return Value(V_NUMBER, 5*a+b)

_builtins = {}
_builtins['ftest'] = _ftest

    
V_NUMBER = 0
V_STRING = 1
V_OPERATOR = 2

class Value():
    def __init__(self, type, val):
        self.type = type
        self.val = val
    def __repr__(self):
        if self.type == V_OPERATOR:
            return "%r"%(NUM2STR[self.val])
        else:
            return "%r"% self.val

def qeval(s):
    ops = []
    stack = []
    tok_iter = iter(tokenize(s))
    next_tok = lambda: next(tok_iter, None)
    while True:
        tok = next_tok()
        if tok is None:
            break
        
        if tok.type == NAME:
            stack.append(Value(V_STRING, tok.val))
            
        elif tok.type == NUMBER:
            stack.append(Value(V_NUMBER, float(tok.val)))
            
        elif tok.type == OPERATOR:
            o = operators[tok.val]
            while ops:
                v = ops[-1].val
                if v == "(":
                    break
                o2 = operators[v]
                if o.p > o2.p or (o.p == o2.p and o2.a == "RIGHT"):
                    break
                stack.append(Value(V_OPERATOR, ops.pop().val))
            ops.append(tok)
            
        elif tok.type == PUSH:
            ops.append(tok)
            
        elif tok.type == POP:
            while ops[-1].type != PUSH:
                stack.append(Value(V_OPERATOR, ops.pop().val))
            ops.pop()
            if ops and ops[-1].type == OPERATOR and ops[-1].val == FUNC:
                #stack.append(Value(V_NUMBER, _builtins[]))
                ops.pop()
                stack.append(Value(V_OPERATOR, FUNC))
            
    while ops:
        stack.append(Value(V_OPERATOR, ops.pop().val))
    #print(stack)
    mstack = []
    for v in stack:
        #print(mstack)
        if v.type == V_NUMBER or v.type == V_STRING:
            mstack.append(v)
        elif v.type == V_OPERATOR:
            op = operators[v.val]
            op.exec(mstack)
        else:
            raise TypeError(v.type)
    return mstack.pop().val

In [11]:
qeval("ftest(1,2)")

TypeError: _ftest() missing 1 required positional argument: 'b'